# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
# file path & name 
file = "../../WeatherPy/Output/cities.csv"

# Read CSV to Dataframe
cities_df = pd.read_csv(file, index_col=False)

In [6]:
# Display dataframe 
cities_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,1,kruisfontein,100,ZA,1580962731,87,-34.00,24.73,64.220,6.77
1,2,victoria,40,HK,1580962684,72,22.29,114.16,69.998,9.30
2,3,mangan,0,IN,1580962732,35,27.52,88.53,42.584,1.17
3,4,port alfred,100,ZA,1580962732,96,-33.59,26.89,68.000,1.79
4,5,gilroy,1,US,1580962732,74,37.01,-121.57,51.998,0.80
...,...,...,...,...,...,...,...,...,...,...
495,496,mbanza-ngungu,74,CD,1580962787,94,-5.25,14.87,66.236,0.89
496,497,paducah,90,US,1580962787,93,37.08,-88.60,37.994,2.10
497,498,port-gentil,60,GA,1580962787,72,-0.72,8.78,82.274,2.17
498,499,leningradskiy,100,RU,1580962787,97,69.38,178.42,-22.990,6.81


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Configure gmaps
gmaps.configure(api_key=g_key)
m = gmaps.Map()

# Use the Lat and Lng as locations and Humidity as the weight
# Extract latitude, longitude from the dataframe and then convert to a list of latitude & longitude to use in gmaps 
lat = []
lng = []
lat = cities_df['Lat'].to_list()
lng = cities_df['Lng'].to_list()
latlng = list(zip(lat,lng))
# Extarct humidity list
humidity = cities_df['Humidity'].to_list()

# Add latitude, longitude & Humididty layer
heatmap_layer = gmaps.Heatmap(locations=latlng, weights=humidity)
m.add_layer(heatmap_layer)
m

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Desired weather conditions:
#     A max temperature lower than 80 degrees but higher than 70
#     Wind speed less than 10 mph
#     Zero cloudiness

cities_selected_df = cities_df.copy()

# Subset max temperature between 70 & 80, wind speed less than 10 and 0 cloudiness
cities_selected_df = cities_selected_df[(cities_selected_df['Max Temp']>70) & \
                                        (cities_selected_df['Max Temp']<80) & \
                                        (cities_selected_df['Wind Speed']<10) & \
                                        (cities_selected_df['Cloudiness']==0)]
# Display selected cities data
cities_selected_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
43,44,bubaque,0,GW,1580962736,61,11.28,-15.83,79.682,1.69
75,76,chuy,0,UY,1580962740,82,-33.70,-53.46,73.922,6.46
85,86,carauari,0,BR,1580962741,93,-4.88,-66.90,74.552,0.06
96,97,jaffna,0,LK,1580962742,70,9.67,80.01,79.034,4.13
156,157,santa maria,0,BR,1580962749,88,-29.68,-53.81,75.200,4.10
173,174,sao filipe,0,CV,1580962751,65,14.90,-24.50,72.230,6.82
193,194,pointe michel,0,DM,1580962753,78,15.25,-61.38,78.854,9.66
196,197,san ignacio,0,BZ,1580962753,94,17.16,-89.07,73.400,3.10
219,220,panaba,0,MX,1580962756,79,21.28,-88.27,75.884,4.61
240,241,tecoanapa,0,MX,1580962758,87,16.52,-98.75,75.326,0.52


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Create a DataFrame to record hotel information. Start from selected cities data 
hotel_df = cities_selected_df[["City", "Country", "Lat", "Lng"]].copy()
# Add an empty column for hotel name
hotel_df["Hotel Name"] = ""

# Display the Hotel DataFrame
hotel_df

,City,Country,Lat,Lng,Hotel Name
43,bubaque,GW,11.28,-15.83,
75,chuy,UY,-33.70,-53.46,
85,carauari,BR,-4.88,-66.90,
96,jaffna,LK,9.67,80.01,
156,santa maria,BR,-29.68,-53.81,
173,sao filipe,CV,14.90,-24.50,
193,pointe michel,DM,15.25,-61.38,
196,san ignacio,BZ,17.16,-89.07,
219,panaba,MX,21.28,-88.27,
240,tecoanapa,MX,16.52,-98.75,


In [9]:
# Search for the Hotel:

api_base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 

# Save criteria & key to search for hotel using google API
api_parms = {"radius":5000, "types":"lodging", "key":g_key}

# Loop through each row in the hotel_df & call API for each row (Use Lat & Lng)
for i, city in hotel_df.iterrows():
    # Latitude - Add to api_parms dictionary 
    api_parms["lat"] = city["Lat"]
    # Longitude - Add to api_parms dictionary
    api_parms["lng"] = city["Lng"]
    
    hotel_name = ''
    try:
        # Call API:
        results = requests.get(api_base_url, params=api_parms)
        # Proceed if success 
        if results.status_code == 200:
            hotels = results.json()
             # get 1st hotel name if 1 or more results found
            if len(hotels["results"]) >=1:
                hotel_name = hotels["results"][0]["name"]
            else:
                print("No hotels found for city: {}".format(city["City"]))
        else:
            print("API call not successful while getting hotel for city: {}".format(city["City"]))
        # for logging - Print the hotels results
        print(hotels)
    except Exception as e:
        print("Error getting hotel for city: {}".format(city["City"]))
        print("Error is {}". format(e))
        

    # if hotel name is '' i.e not retreived - Set as "*** NOT FOUND ***"
    if hotel_name == '':
        hotel_name = '*** NOT FOUND ***'
    # Add to hotel_df
    hotel_df.loc[i, "Hotel Name"] = hotel_name

# Done looping through the DataFrame - Display update hotel_df with hotel name
hotel_df

No hotels found for city: bubaque
{'error_message': 'The provided API key is invalid.', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
No hotels found for city: chuy
{'error_message': 'The provided API key is invalid.', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
No hotels found for city: carauari
{'error_message': 'The provided API key is invalid.', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
No hotels found for city: jaffna
{'error_message': 'The provided API key is invalid.', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
No hotels found for city: santa maria
{'error_message': 'The provided API key is invalid.', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
No hotels found for city: sao filipe
{'error_message': 'The provided API key is invalid.', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
No hotels found for city: pointe michel
{'error_message': 'T

,City,Country,Lat,Lng,Hotel Name
43,bubaque,GW,11.28,-15.83,*** NOT FOUND ***
75,chuy,UY,-33.70,-53.46,*** NOT FOUND ***
85,carauari,BR,-4.88,-66.90,*** NOT FOUND ***
96,jaffna,LK,9.67,80.01,*** NOT FOUND ***
156,santa maria,BR,-29.68,-53.81,*** NOT FOUND ***
173,sao filipe,CV,14.90,-24.50,*** NOT FOUND ***
193,pointe michel,DM,15.25,-61.38,*** NOT FOUND ***
196,san ignacio,BZ,17.16,-89.07,*** NOT FOUND ***
219,panaba,MX,21.28,-88.27,*** NOT FOUND ***
240,tecoanapa,MX,16.52,-98.75,*** NOT FOUND ***


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map